# Import packages

In [1]:
import os
import sys

import numpy as np
import torch
import os.path as osp
import losses as L
import augmentors as A
import torch.nn.functional as F
import torch_geometric.transforms as T

from tqdm import tqdm
from torch.optim import Adam
from eval import get_split, LREvaluator
from models import DualBranchContrast
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


# Dependencies

## Define Classes and Training Pipeline

In [2]:
class GConv(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, activation, num_layers):
        super(GConv, self).__init__()
        self.activation = activation()
        self.layers = torch.nn.ModuleList()
        self.layers.append(GCNConv(input_dim, hidden_dim, cached=False))
        for _ in range(num_layers - 1):
            self.layers.append(GCNConv(hidden_dim, hidden_dim, cached=False))

    def forward(self, x, edge_index, edge_weight=None):
        z = x
        for i, conv in enumerate(self.layers):
            z = conv(z, edge_index, edge_weight)
            z = self.activation(z)
        return z


class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor, hidden_dim, proj_dim):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

        self.fc1 = torch.nn.Linear(hidden_dim, proj_dim)
        self.fc2 = torch.nn.Linear(proj_dim, hidden_dim)

    def forward(self, x, edge_index, edge_weight=None):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index, edge_weight)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index, edge_weight)
        z = self.encoder(x, edge_index, edge_weight)
        z1 = self.encoder(x1, edge_index1, edge_weight1)
        z2 = self.encoder(x2, edge_index2, edge_weight2)
        return z, z1, z2

    def project(self, z: torch.Tensor) -> torch.Tensor:
        z = F.elu(self.fc1(z))
        return self.fc2(z)


def train(encoder_model, contrast_model, data, optimizer):
    encoder_model.train()
    optimizer.zero_grad()
    z, z1, z2 = encoder_model(data.x, data.edge_index, data.edge_attr)
    h1, h2 = [encoder_model.project(x) for x in [z1, z2]]
    loss = contrast_model(h1, h2)
    loss.backward()
    optimizer.step()
    return loss.item()


def test(encoder_model, data):
    encoder_model.eval()
    z, _, _ = encoder_model(data.x, data.edge_index, data.edge_attr)
    split = get_split(num_samples=z.size()[0], train_ratio=0.1, test_ratio=0.8)
    result = LREvaluator()(z, data.y, split)
    return result

## Load Dataset

In [3]:
path = osp.join(osp.expanduser('~'), 'datasets')
dataset = Planetoid(path, name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]
N = data.num_nodes

## Construct Model

In [6]:
def run_model(aug1, aug2):
    gconv = GConv(input_dim=dataset.num_features, hidden_dim=32, activation=torch.nn.ReLU, num_layers=2)
    # .to(device)
    encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2), hidden_dim=32, proj_dim=32)
    # .to(device)
    contrast_model = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='L2L', intraview_negs=True)
    # .to(device)

    optimizer = Adam(encoder_model.parameters(), lr=0.01)

    with tqdm(total=1000, desc='(T)') as pbar:
        for epoch in range(1, 1001):
            loss = train(encoder_model, contrast_model, data, optimizer)
            pbar.set_postfix({'loss': loss})
            pbar.update()

            if epoch % 200 == 0:
                print(f'epoch={epoch}, loss={loss}')


    test_result = test(encoder_model, data)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')

# Experiments by Proposed Model

## (Proposed) Running Example for 'FM+TopK' on Cora Dataset

In [5]:
r = 0.3
k = round(N * r)
print(f'=================================================================')
print(f'k_ratio={r}, TopKSubgraph_k={k}')
aug1 = A.Compose([A.FeatureMasking(pf=0.3),
                  A.TopKSubgraph(N, k=k)])
aug2 = A.Compose([A.FeatureMasking(pf=0.3),
                  A.TopKSubgraph(N, k=k)])
run_model(aug1, aug2)

k_ratio=0.3, TopKSubgraph_k=812


(LR): 100%|██████████| 1000/1000 [00:00<00:00, best test F1Mi=0.809, F1Ma=0.772]

(E): Best test F1Mi=0.8088, F1Ma=0.7724


## (Proposed) Running Example for 'FM+Khop' on Cora Dataset

In [7]:
r = 0.2
k = round(N * r)
num_hop = 4
print(f'=================================================================')
print(f'k_ratio={r}, k={k}, KhopSubgraph_num_hops={num_hop}')
aug1 = A.Compose([A.FeatureMasking(pf=0.3),
                  A.KhopSubgraph(N, k=k, num_hops=num_hop)])
aug2 = A.Compose([A.FeatureMasking(pf=0.3),
                  A.KhopSubgraph(N, k=k, num_hops=num_hop)])
run_model(aug1, aug2)

k_ratio=0.2, k=542, KhopSubgraph_num_hops=4


(T):  20%|██        | 200/1000 [01:39<06:53,  1.93it/s, loss=5.51]

epoch=200, loss=5.5083088874816895


(T):  40%|████      | 400/1000 [03:19<04:48,  2.08it/s, loss=5.1] 

epoch=400, loss=5.104148864746094


...

epoch=800, loss=4.709908485412598


(T): 100%|██████████| 1000/1000 [08:25<00:00,  1.98it/s, loss=4.6]


epoch=1000, loss=4.598392963409424


(LR): 100%|██████████| 1000/1000 [00:00<00:00, best test F1Mi=0.735, F1Ma=0.707]

(E): Best test F1Mi=0.7353, F1Ma=0.7071


## Runing Example for Augmentation Schemes Used in GRACE

In [8]:
pe = 0.3
print('====================================================================================')
print(f'Experimental result for baseline --- GRACE')
aug1 = A.Compose([A.EdgeRemoving(pe=pe), A.FeatureMasking(pf=0.3)])
aug2 = A.Compose([A.EdgeRemoving(pe=pe), A.FeatureMasking(pf=0.3)])
run_model(aug1, aug2)

aug=[A.EdgeRemoving(pe=0.3), A.FeatureMasking(pf=0.3)]


(T):  20%|██        | 200/1000 [01:36<05:52,  2.27it/s, loss=5.76]

epoch=200, loss=5.760686874389648


(T):  40%|████      | 400/1000 [03:12<04:47,  2.09it/s, loss=5.36]

epoch=400, loss=5.358218193054199


...

epoch=800, loss=4.932802200317383


(T): 100%|██████████| 1000/1000 [07:38<00:00,  2.18it/s, loss=4.86]


epoch=1000, loss=4.859668254852295


(LR): 100%|██████████| 1000/1000 [00:00<00:00, best test F1Mi=0.739, F1Ma=0.701]

(E): Best test F1Mi=0.7390, F1Ma=0.7011


## Running Example for Augmentation Schemes Used in GraphCL

In [9]:
print(f'=================================================================')
print(f'Experimental result for baseline --- GraphCL')
aug1 = A.Identity()
aug2 = A.RandomChoice([A.RWSampling(num_seeds=1000, walk_length=10),
                   A.NodeDropping(pn=0.1),
                   A.FeatureMasking(pf=0.1),
                   A.EdgeRemoving(pe=0.1)], 1)
run_model(aug1, aug2)

Experimental result for baseline --- GraphCL


(T):  20%|██        | 200/1000 [01:24<05:28,  2.43it/s, loss=5.21]

epoch=200, loss=5.208513259887695


(T):  40%|████      | 400/1000 [02:48<04:07,  2.43it/s, loss=4.8] 

epoch=400, loss=4.804235458374023


...

epoch=800, loss=4.471333026885986


(T): 100%|██████████| 1000/1000 [07:14<00:00,  2.30it/s, loss=4.4]


epoch=1000, loss=4.403059005737305


(LR): 100%|██████████| 1000/1000 [00:00<00:00, best test F1Mi=0.647, F1Ma=0.634]

(E): Best test F1Mi=0.6471, F1Ma=0.6339


## Running Example for Augmentation Schemes Used in MVGRL

In [10]:
print(f'=================================================================')
print(f'Experimental result for baseline --- MVGRL')
aug1 = A.Identity()
aug2 = A.PPRDiffusion(alpha=0.2)
run_model(aug1, aug2)

Experimental result for baseline --- MVGRL


(T):  20%|██        | 200/1000 [02:25<09:36,  1.39it/s, loss=5.66]

epoch=200, loss=5.655943870544434


(T):  40%|████      | 400/1000 [04:57<07:51,  1.27it/s, loss=4.95]

epoch=400, loss=4.946862697601318


...

epoch=800, loss=4.454775810241699


(T): 100%|██████████| 1000/1000 [12:08<00:00,  1.37it/s, loss=4.36]


epoch=1000, loss=4.362445831298828


(LR): 100%|██████████| 1000/1000 [00:00<00:00, best test F1Mi=0.57, F1Ma=0.523]


(E): Best test F1Mi=0.5699, F1Ma=0.5228
